In [55]:
from keras.utils import to_categorical


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

pd.set_option('display.max_columns', None)

In [44]:
complaints_df = pd.read_pickle('../pickle/complaints_v2.pkl')
reports_df = pd.read_pickle('../pickle/reports.pkl')

#add a url feature to match with reports_df
complaints_df['url'] = complaints_df.apply(lambda x: x.complaints['url'],axis=1)
reports_df['url'] = reports_df.apply(lambda x: x.complaints['url'],axis=1)

In [45]:
df = reports_df.merge(complaints_df, on='url')
df_t = df.drop_duplicates('url').copy()

In [46]:
df_t = df_t[['complaint_detail','enforcement']]
df_t['enforcement'] = df_t['enforcement'].apply(lambda x: 1 if type(x) == dict else 0 )

In [47]:
df_t

complaint_detail  enforcement
0              VERY STRONG ROTTEN EGGS SMELL IN THE AIR.            0
8      CAR X                               DUMPING OI...            0
9      HAPPY VILLAGE BAR                   VERY LOUD ...            0
11     JEWEL COMPLEX                       DEBRIS. TH...            0
12     VACANT LOT                          511 UNIT R...            0
...                                                  ...          ...
83569  PUBLIC WAY                          5 GALLON C...            0
83570  RESIDENTIAL BLDG.                   FLY DUMPIN...            0
83571  RESIDENTIAL AREA                    RAW GARBAG...            0
83573  NEW CONSTRUCTION SITE               EARLY MORN...            0
83574  GOLDEN AUTO BODY REBUILDERS         JUNKYARD W...            0

[31762 rows x 2 columns]

In [48]:
df_t.complaint_detail = df_t.complaint_detail.apply(str)
df_t = df_t.drop_duplicates('complaint_detail')

In [49]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df_t.complaint_detail)
sequences = tokenizer.texts_to_sequences(df_t.complaint_detail)

In [50]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 25829 unique tokens.


In [51]:
print(sum([len(i) for i in sequences])/83574)

8.932873860291478


In [40]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [57]:
labels = to_categorical(np.asarray(df_t.enforcement))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (31762, 350)
Shape of label tensor: (30806, 2)
